In [25]:
from urllib.request import HTTPBasicAuthHandler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import mutual_info_classif



# Load the data
df = pd.read_csv("Chinook_Employee_Joins_Aggregated_Nums.csv")

print(df.head())

   EmployeeId Employee_FirstName Employee_LastName Sex Employee_HireDate  \
0           3               Jane           Peacock   F        2002-04-01   
1           4           Margaret              Park   F        2003-05-03   
2           5              Steve           Johnson   M        2003-10-17   
3          19              Doris           Hartwig   M        2007-10-09   
4          16             Taylor           Maxwell   M        2007-09-08   

  Employee_BirthDate  Employee_Age  ReportsTo        Employee_Role  \
0         1973-08-29            51        2.0  Sales Support Agent   
1         1947-09-19            77        2.0  Sales Support Agent   
2         1965-03-03            59        2.0  Sales Support Agent   
3         1955-11-03            69        2.0  Sales Support Agent   
4         1955-10-31            69        2.0  Sales Support Agent   

   TotalInvoices  TotalRevenue  AvgRevenue  
0            146        833.04    5.705753  
1            140        775.40  

In [26]:
# Convert 'Employee_HireDate' to a datetime object
df['Employee_HireDate'] = pd.to_datetime(df['Employee_HireDate'])

# Calculate tenure in years
current_date = pd.Timestamp.now()
df['Tenure'] = (current_date - df['Employee_HireDate']).dt.days / 365

# Format AvgRevenue to 2 decimal places
df['AvgRevenue'] = df['AvgRevenue'].round(2)

# Fill any remaining NaN values with 0
df[['TotalRevenue', 'AvgRevenue', 'ReportsTo']] = df[['TotalRevenue', 'AvgRevenue', 'ReportsTo']].fillna(0)

# Calculate the total average per year (TotalRevenue / Tenure)
df['AnnualRevenue'] = (df['TotalRevenue'] / df['Tenure']).round(2)

# Round tenure to the nearest integer
df[['Tenure', 'ReportsTo']] = df[['Tenure', 'ReportsTo']].round().astype(int)

In [27]:
# Remove outliers where 'Employee_Role' is not 'Sales Support Agent' and employeeId is 3,4,5
df2 = df[df['Employee_Role'] == 'Sales Support Agent']
df2[~df2['EmployeeId'].isin([3, 4, 5])]

,EmployeeId,Employee_FirstName,Employee_LastName,Sex,Employee_HireDate,Employee_BirthDate,Employee_Age,ReportsTo,Employee_Role,TotalInvoices,TotalRevenue,AvgRevenue,Tenure,AnnualRevenue
3,19,Doris,Hartwig,M,2007-10-09,1955-11-03,69,2,Sales Support Agent,37,181.92,4.92,17,10.51
4,16,Taylor,Maxwell,M,2007-09-08,1955-10-31,69,2,Sales Support Agent,31,171.87,5.54,17,9.88
5,54,Kirk,Koenigsbauer,M,2008-07-15,1984-09-06,40,2,Sales Support Agent,29,170.77,5.89,17,10.32
6,29,Paul,Komosinski,M,2008-07-04,1980-06-13,44,2,Sales Support Agent,25,159.16,6.37,17,9.60
7,40,Jinghao,Liu,F,2008-07-08,1988-09-05,36,2,Sales Support Agent,24,156.24,6.51,17,9.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,291,Jae,Pak,F,2011-12-29,1967-10-16,57,2,Sales Support Agent,0,0.00,0.00,13,0.00
292,242,Frank,Pellow,M,2009-08-23,1951-12-11,73,2,Sales Support Agent,0,0.00,0.00,15,0.00
293,254,Krishna,Sunkammurali,F,2009-09-13,1971-04-05,53,2,Sales Support Agent,0,0.00,0.00,15,0.00
294,231,Mary,Baker,F,2009-07-25,1986-04-19,38,2,Sales Support Agent,0,0.00,0.00,16,0.00


In [28]:
features = ['EmployeeId', 'Employee_Role', 'Sex', 'Employee_Age', 'Tenure',
            'TotalInvoices', 'TotalRevenue', 'AvgRevenue', 'AnnualRevenue']

df_final = df2[features]
df_final = df_final.copy()

# Define performance labels based on quantiles of 'TotalRevenue'
quantile_labels = ['Low Performer', 'Average Performer', 'High Performer']
df_final['Performance_Label'] = pd.qcut(df_final['TotalRevenue'], q=3, labels=quantile_labels, duplicates='drop')

# Create a dictionary to map labels to desired numbers
label_mapping = {'Low Performer': 0, 'Average Performer': 1, 'High Performer': 2}

# Map labels to numbers
df_final['Performance_Label_Encoded'] = df_final['Performance_Label'].map(label_mapping)

# Display the first few rows of the filtered DataFrame
print(df_final.head())

   EmployeeId        Employee_Role Sex  Employee_Age  Tenure  TotalInvoices  \
0           3  Sales Support Agent   F            51      23            146   
1           4  Sales Support Agent   F            77      22            140   
2           5  Sales Support Agent   M            59      21            126   
3          19  Sales Support Agent   M            69      17             37   
4          16  Sales Support Agent   M            69      17             31   

   TotalRevenue  AvgRevenue  AnnualRevenue Performance_Label  \
0        833.04        5.71          36.48    High Performer   
1        775.40        5.54          35.65    High Performer   
2        720.16        5.72          33.82    High Performer   
3        181.92        4.92          10.51    High Performer   
4        171.87        5.54           9.88    High Performer   

  Performance_Label_Encoded  
0                         2  
1                         2  
2                         2  
3                   

In [42]:
# Calculate correlation / mutual information between input features and output label
input_features = ["TotalInvoices", "AvgRevenue","AnnualRevenue","Employee_Age"]
mi_scores = mutual_info_classif(df_final[input_features] ,df_final[['Performance_Label_Encoded']])
feature_importances = pd.DataFrame({
    "Feature": input_features,
    "Mutual Information Score": mi_scores})
print(feature_importances)

         Feature  Mutual Information Score
0  TotalInvoices                  0.912111
1     AvgRevenue                  0.646851
2  AnnualRevenue                  1.067710
3   Employee_Age                  0.166545


c:\Users\User\anaconda3\envs\py39-basic-sk-pandas\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
